This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Static clutter map

Detects the average level of the returned signal. It is recommended to use at least one month of data.

In [1]:
%load_ext line_profiler
%load_ext memory_profiler


import glob
import os
import warnings

import numpy as np
import matplotlib
import matplotlib.pyplot as pl
warnings.filterwarnings('ignore')
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()
import xarray as xr

import wradlib as wrl
import wradlib.clutter as clutter
import wradlib.util as util
import wradlib.vis as vis

In [2]:
import sys
import os
import psutil
process = psutil.Process(os.getpid())
import xarray

### Load sweep data

In [3]:
var="TH"
nod="bewid"
#var="DBZH"
wrl_path = util.get_wradlib_data_path()

if var == "TH":
    basename = '*dBuZ*.vol.h5'
if var == "DBZH":
    basename = '*dBZ*.vol.h5'

date="20190901"
basename = "{}{}".format(date,basename)



files = os.path.join(wrl_path,'hdf5',nod,basename)
files = glob.glob(files)
files.sort()
files = files[0:2]
pvol = wrl.io.xarray.OdimH5(files,standard="cf-mandatory",mask_and_scale=False,dim0='azimuth',chunks=50,fix_coords=['time'])

print(process.memory_info().rss)
print(pvol["sweep_1"].TH.variable._data)
sys.exit()

Concat: 100%|██████████| 2/2 [00:00<00:00, 210.34 Timesteps/s]


194699264
dask.array<getitem, shape=(2, 360, 500), dtype=uint8, chunksize=(1, 50, 50)>


SystemExit: 


### Select and decode data

In [ ]:
maxdist = 80
maxelev = 1
elangles = pvol.root.sweep_fixed_angle
select = pvol.root.sweep_group_name[elangles < 1]
select = select.values
for sweep in select:
    print(sweep)
    pvol[sweep] = pvol[sweep].sel(range=slice(0,maxdist*1E3))
startdate = str(pvol[sweep].time.values[0])[:10]
enddate = str(pvol[sweep].time.values[-1])[:10]

print(process.memory_info().rss)




### Get the static clutter levels

In [ ]:
import importlib
importlib.reload(clutter)
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

import time

for sweep in select:
    start = time.time()
    vardata = pvol[sweep][var]
    pvol[sweep] = clutter.static_clutter_map(vardata)
    print(time.time()-start)
print(process.memory_info().rss)

### Plot the results

In [ ]:
source = pvol.root.attrs["institution"]
source = source.split(",")
nod = None
for s in source:
    if "WMO" in s:
        wmo = s.split(":")[1]
        source=wmo
    if "NOD" in s:
        nod = s.split(":")[1]
        source=nod


datestr = "{}-{}".format(startdate, enddate)
diststr = "{}km".format(maxdist)


pvol.georeference()        
for sweep in select:
    elangle = str(pvol[sweep].fixed_angle.values)
    print(elangle)
    params = [source, var, elangle, diststr, datestr]
    titlestr = " ".join(params)
    pvol[sweep] = xr.decode_cf(pvol[sweep])
    
    da = pvol[sweep]["static_clutter_level"]
    
    fig = pl.figure(figsize=(16,16))
    fig.suptitle(titlestr)
    pl.axis("off")
    pm = da.wradlib.plot_ppi(fig=fig, vmin=0, vmax=60)
    ax = pl.gca()
    cb = pl.colorbar(pm, ax=ax, label="static_clutter_level")
    filename = "_".join(params)
    filename = "{}_static_clutter_level.png".format(filename)
    pl.savefig(filename, bbox_inches='tight', format="png")

    da = pvol[sweep]["static_clutter_resolution"]
    fig = pl.figure(figsize=(16,16))    
    fig.suptitle(titlestr)
    pl.axis("off")
    pm = da.wradlib.plot_ppi(fig=fig, vmin=0, vmax=10)
    ax = pl.gca()
    cb = pl.colorbar(pm, ax=ax, label="static_clutter_resolution")
    filename = "_".join(params)
    filename = "{}_static_clutter_resolution.png".format(filename)
    pl.savefig(filename, bbox_inches='tight')
        
        